In [19]:
import json
import argparse
import os
import eval_utils as EU
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from gensim.models import KeyedVectors

In [ ]:
embedding_storage = {"node2vec": '../node2vec/emb/', "ProNE": '../ProNE/emb/'}

with open("../embedding_config.json", "r") as jsonfile:
    embeddding_config = json.load(jsonfile)
num_bins = embeddding_config["num_bins"]
test_size = embeddding_config["test_size"]

In [2]:
task = "genes"
with open("../data/data_config.txt", "r") as jsonfile:
    data_config = json.load(jsonfile)
with open("../data/strategies/" + task + ".txt", "r") as jsonfile:
    strategies = json.load(jsonfile)
config = data_config[task]
location = config["location"]
target_file = config["target_file"]
location_processed = config["location_processed"]
target_column = config["target_column"]

In [16]:
small_table = pd.read_csv(os.path.join("../", location + target_file),
                         sep=',',
                         encoding='latin')
full_table = pd.read_csv(os.path.join("../", location + "Genes.csv"),
                         sep=',',
                         encoding='latin')

Y = small_table[target_column]
Y = pd.Categorical(Y).codes

In [27]:
for col in full_table.columns: 
    print(full_table[col].dtype)
    if full_table[col].dtype in ["str", "object"]:
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(full_table[col])
        full_table[col] = integer_encoded
        print(integer_encoded)
full_table["res"] = Y

int64
int64
int64
int64
int64
int64
int64
int64


ValueError: Length of values (862) does not match length of index (4346)

In [37]:
label_encoder = LabelEncoder()
small_table['GeneID'] = label_encoder.fit_transform(small_table['GeneID'])
small_table

,GeneID,Localization
0,0,cytoplasm
1,1,cytoplasm
2,2,cytoskeleton
3,3,cytoplasm
4,4,cytoplasm
...,...,...
857,857,cytoplasm
858,858,cytoplasm
859,859,nucleus
860,860,cytoplasm


In [43]:
X = pd.merge(small_table, full_table, left_on = "GeneID", right_on = 'GeneID', how = "left").drop_duplicates(subset=["GeneID"])

In [49]:
X.drop(columns="Localization"), Y

(      GeneID  Essential  Class  Complex  Phenotype  Motif  Chromosome  \
 0          0          2      7       47          0    176           1   
 4          1          3      1        1          0      0           1   
 12         2          3      1       12          6      4           1   
 18         3          3      4       11          3      2           1   
 20         4          2      7        1          4    159           1   
 ...      ...        ...    ...      ...        ...    ...         ...   
 4336     857          3      0        1          6      0          11   
 4338     858          3      0        1          1      0           7   
 4340     859          3      0        1          3      0           4   
 4342     860          3      0        1          0     97           4   
 4344     861          3      0        1          8    185          12   
 
       Function  
 0            4  
 4            1  
 12           0  
 18           0  
 20           0  
 .

In [69]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

tests = train_test_split(X, Y, test_size = 0.2)
confusion_matrix(tests[3], tests[3]).diagonal()

array([13,  1, 42, 11,  8,  6, 75,  1,  8,  3,  5])

In [71]:
117/confusion_matrix(tests[3], tests[3]).diagonal().sum()

0.6763005780346821